# The architecture of a Spark Application
**The Spark Driver**
This is the process in the driver's seat of your spark application. It is the controller of the execution of a spark application and maintains all of the state of the spark cluster(the state and tasks of the executors). It must interface wityh the cluster manager in order to get physical resources and launmch executors. At the end of the day, this is just a process ona  physical machine that is responsible for maintaining the state of the application running on the cluster.

**The Spark Executors**
Spark executors are the process that perform the tasks assigned by the spark driver. Executors have one core responsibility: take the tasks assigned by the driver, run them, and report back their state(success or failure) and results. Each spark application has its own separate executor process

**The cluster manager**
The cluster manager is responsible for maintaining a cluster of machines that will run your spark application. A cluster manager will have its own driver(sometimes called master) and 'worker' abstractions. The core difference is that they are tied to physical machines rather than processes.

<img src="/home/kevin/Desktop/Big-Data-with-Pyspark/images/01_cluster_driver.png">


The machine on the left is the cluster manager driver node. The circles represent daemon processes running on and managing each of the individual worker nodes. There is no spark application running as of yet - these are just the processes from the cluster manager. When it comes time to actually run a spark application, we request resources from the cluster manager to run it. Depending on how the application is configured, this can include a place to run the spark driver or might just be resources for the executors for our spark application. 
Spark currently supports 3 cluster managers: a simple built-in standalone cluster manager, apache mesos, and hadoop yarn. 

## Execution Modes
An execution mode gives you the power to determine the aforementioned resources that are physically located when you go run your application. You have 3 modes to choose from:
* Cluster mode
* Client mode
* Local mode


### Cluster mode
This is the most common way of running a spark application. In cluster mode, a user submits a pre-compiled python script to a cluster manager. The cluster manager then launches the driver process on a worker node inside the cluster, in addition to the executor process

<img src="/home/kevin/Desktop/Big-Data-with-Pyspark/images/02_cluster_mode.png">

### client mode
It is nearly the same as the cluster mode except that the spark driver remains on the client machine that submitted the application. This means that the client machine is responsible for maintaining the spark driver process, and the cluster manager maintains the executor processes. In the following figure, we are running tyhe spark application from a machine that is not colocated on the cluster. These machines are commonly referred to as gateway machines or edge nodes. In the figure, you can see tha5t the driver is running on a machine outside the cluster but that the workers are located on machines in the cluster

<img src="/home/kevin/Desktop/Big-Data-with-Pyspark/images/03_client_mode.png">

### local mode
It runs the entire spark application on a single machine. It achives parallelism through threads on that single machine. This is a common way to olearn spark, to test your applications, or experiment iteratively with local development

# The Life Cycle of a Spark Application (Outside Spark)

## Client Request
The first step is for you to submit an actual application. That will be pre-compiled JAR or library. At this point, you are executing code on your local machine and you're going to make a request to the cluster manager driver node. Heree we are explicitly asking for resources for the spark driver process only. We assume the cluster manager accepots this offer and places the driver onto a node in the cluster. The client process that submitted the original job exits and the application is off and running on the cluster.

<img src="/home/kevin/Desktop/Big-Data-with-Pyspark/images/04_client_request.png">

To do this, we run the following code in the terminal
```
.bin/spark-submit \
--class <main-class>\
--deploy-mode cluster \
--conf <key>=<value> \
... # other options
<application-jar> \
[application arguments]
```

## Launch
Now that the driver process has been placed on the cluster, it begins running user code. This code musty include a SparkSession that initiualizes a spark cluster (driver + executors). The SparkSession will subsequently communicate with the cluster manager (darker line), asking it to launch spark executor process across the cluster (lighter lines). The number of executors and their relevant configurations are set by the user via the command-line arguments in the original spark-submit call
The cluster manager responds by launching the executopr processs (assuming all goes well) and sends the relevant information about theiur location to the driver process. Afetr everything ios hooked up correctly, we have a spark-cluster

<img src="/home/kevin/Desktop/Big-Data-with-Pyspark/images/05_spark_launch.png">


## Execution
Now that we have a spark cluster, spark executes its code. The driver and the workere communicate among themselves, executing code and moving data around. The driver schedules tasks onto each worker, and each worker responds with the status of those tasks and sucess or failure.

<img src="/home/kevin/Desktop/Big-Data-with-Pyspark/images/06_application_execution.png">


## Completion
After spark application completes, the driver process exits with either sucess or failure. The cluster manager then shuts down tyhe executors in that spark cluster for the driver

# The Life Cycle of a Spark Application (Inside Spark)


## The SparkSession
The first step is to create a sparksession. Some of your legacy code might use the new SparkContext pattern. This should be avoided in favor of the builder method on the Sparsession, which more robustly instantiates the spark and sql contexts and ensures there is no context conflict, given that there might be multiple libraries trying to create a session in the same spark application


In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
                    .master('local')\
                    .appName('Word Count')\
                    .config('spark.some.config.option', 'some-value')\
                    .getOrCreate()

After you have a SparkSession, you should be able to run your spark code. From the sparksession, you can access all of low-level and legacy contexts and configurations accordingly.

### The SparkContext
This is an object within the SparkSession that represents the connection with the spark cluster. This class is how you communicate with some of spark's lower-level APIs such as RDDs. It is commonly stored as the variable sc in older examples anbd docs. Through a SparkContext, you can create RDDs, accumulators and broadcast variables, and you can run code on the cluster.


## Logical Instructions
We will take a simpler approach. We are going to do a three-step job using a simple DataFrame, we'll repartition it, perform a value-by-value manipulation, and then aggregate some values and collect the final result

In [6]:
df1 = spark.range(2, 10000000, 2)
df2 = spark.range(2, 10000000, 4)
step1 = df1.repartition(5)
step12 = df2.repartition(6)
step2 = step1.selectExpr('id * 5 as id')
step3 = step2.join(step12, ['id'])
step4 = step3.selectExpr('sum(id)')

step4.collect()
print('***'*10)
step4.explain()

******************************
== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=true
+- == Final Plan ==
   *(7) HashAggregate(keys=[], functions=[sum(id#30L)])
   +- ShuffleQueryStage 4
      +- Exchange SinglePartition, ENSURE_REQUIREMENTS, [plan_id=264]
         +- *(6) HashAggregate(keys=[], functions=[partial_sum(id#30L)])
            +- *(6) Project [id#30L]
               +- *(6) SortMergeJoin [id#30L], [id#24L], Inner
                  :- *(4) Sort [id#30L ASC NULLS FIRST], false, 0
                  :  +- AQEShuffleRead coalesced
                  :     +- ShuffleQueryStage 2
                  :        +- Exchange hashpartitioning(id#30L, 200), ENSURE_REQUIREMENTS, [plan_id=135]
                  :           +- *(3) Project [(id#22L * 5) AS id#30L]
                  :              +- ShuffleQueryStage 0
                  :                 +- Exchange RoundRobinPartitioning(5), REPARTITION_BY_NUM, [plan_id=68]
                  :                    +- *(1) Range (2, 10000000, 

## A Spark Job
In general, there should be one spark job for one action. Actions always return results. Each job breaks down into a series of stages, the nnumber of which depends on how many shu8ffle operations need to take place.
This job breaks down into the following stages and tasks:
* Stage 1 with 8 tasks
* stage 2 with 8 tasks
* Stage 3 with 6 tasks
* Stage 4 with 5 tasks
* Stage 5 with 200 tasks
* Stage 6 with 1 task

## Stages
Stages in spark represent group of tasks that can be executed together to compute the same operation on multiple machines. In general, spark will try to pack as much work as possible(as many transformations as possible) into the same stage, but the engine starts new stages after operations called shuffles. A shuffle represents a physical repartitioning of the data - for example, sorting a dataframe, or grouping data that was loaded from a file by key. This type of repartitioning requires coordination across executors to move data around. Spark starts a new stage after each shuffle, and keeps track of what order the stages must run in to compute the final result

In thr job we looked at earlier, the nfirst two steps correspond to the range that you perform in order to create your DataFrame. By default, when you create a DataFrame with range, it has 8 partitions. The next step is to repartition. This changes the number of partitions byu shuffling the data. These dataframes are shuffled into six partitions and five partitions, corresponding to the number of tasks in stages 3 and 4

Stages 3 and 4 perform on each of those DataFrames and the end of the stage represents the join(a shuffle). Suddnly we have 200 tasks. This is because of a Spark SQL configuration. The spark.sql.shuffle.partitions default value is 200, which means that there is a shuffle performed during execution, it outputs 200 shuffle partitions by default. 